In [6]:
import pandas as pd
import ollama
import json
import re
from tqdm.auto import tqdm

In [7]:
# -------------------------------------------------
# 1. Load data and add missing 'id'
# -------------------------------------------------
df = pd.read_csv('data.csv')
df = df.reset_index(drop=True)           # make sure index is clean
df['id'] = df.index                       # ← adds id 0, 1, 2, ..., 199
documents = df.to_dict(orient='records')

In [8]:
# -------------------------------------------------
# 2. Robust JSON extraction function
# -------------------------------------------------
def safe_json_extract(text):
    """Extracts a valid {"questions": [...]} dict even if model is messy"""
    text = text.strip()
    
    # Try direct json load first (fast path)
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    
    # Fallback 1: look for JSON inside markdown ```json ... ```
    json_match = re.search(r"```(?:json)?\s*({.*?})\s*```", text, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(1))
        except json.JSONDecodeError:
            pass
    
    # Fallback 2: extract anything that looks like ["...", "..."]
    list_match = re.search(r'\[\s*(?:"[^"]*"(?:\s*,\s*"[^"]*")*)\s*\]', text)
    if list_match:
        try:
            questions_list = json.loads(list_match.group(0))
            return {"questions": questions_list}
        except json.JSONDecodeError:
            pass
    
    # Last resort: split by quotes (very permissive)
    questions = re.findall(r'"([^"]{10,})"', text)  # at least 10 chars to avoid garbage
    if len(questions) >= 3:  # we need at least ~5 questions
        return {"questions": questions[:5]}
    
    raise ValueError(f"Could not extract questions from: {text[:200]}...")

In [9]:
prompt_template = """
You emulate a user of our nutrition assistant application.
Formulate 5 questions this user might ask based on a provided recipe.
Make the questions specific to this recipe.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

recipe_name: {recipe_name}
meal_type: {meal_type}
dietary_category: {dietary_category}
ingredients: {ingredients}
equipment_needed: {equipment_needed}
nutritional_information: {nutritional_information}
instructions: {instructions}
allergens: {allergens}
prep_time_minutes: {prep_time_minutes}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [10]:
# -------------------------------------------------
# 3. Generate questions safely
# -------------------------------------------------
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    response = ollama.chat(
        model='qwen2:1.5b',
        messages=[{"role": "user", "content": prompt}]
    )
    raw = response['message']['content']
    return safe_json_extract(raw)


In [11]:
# -------------------------------------------------
# 4. Main loop (now 100% safe)
# -------------------------------------------------
results = {}

for doc in tqdm(documents, desc="Generating questions"):
    doc_id = doc['id']
    
    if doc_id in results:
        continue
    
    try:
        parsed = generate_questions(doc)
        results[doc_id] = parsed["questions"]
    except Exception as e:
        print(f"\nFailed on doc {doc_id}: {e}")
        results[doc_id] = []  # or skip / retry later

Generating questions:   0%|          | 0/200 [00:00<?, ?it/s]


Failed on doc 60: 'questions'

Failed on doc 94: 'questions'

Failed on doc 108: list indices must be integers or slices, not str


In [12]:
# -------------------------------------------------
# 5. Save ground-truth file
# -------------------------------------------------
final_results = []
for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

df_results = pd.DataFrame(final_results, columns=['id', 'question'])
df_results.to_csv('ground-truth-retrieval.csv', index=False)

print(f"Done! Generated {len(final_results)} question-document pairs.")
print("Saved to ground-truth-retrieval.csv")

Done! Generated 807 question-document pairs.
Saved to ground-truth-retrieval.csv
